# Predicting shots made per game by Kobe Bryant

In this lab you'll be using regularization techniques Ridge, Lasso, and Elastic Net to try and predict well how many shots Kobe Bryant made per game in his career.

---

### 1. Load packages and data

In [1]:
import numpy as np
import pandas as pd
import patsy

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.cross_validation import cross_val_score

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

/Users/basilbeirouti/miniconda3/envs/dsienv/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
kobe = pd.read_csv('../starter/kobe_superwide_games.csv')

---

### 2. Examine the data

- How many columns are there?
- Infer what the observations (rows) and columns represent.
- Why is this data that regularization might be particularly useful for?

In [3]:
print 'Columns:', len(kobe.columns)

Columns: 645


In [4]:
print kobe.columns[0:20]

# The columns are various statistics for each game. 
# There is a column SHOTS_MADE that will be our target variable for prediction
# This is good for regularization because there are so many columns (feature selection)
# and many of the columns represent similar things (multicollinearity)

Index([u'SHOTS_MADE', u'AWAY_GAME', u'SEASON_OPPONENT:atl:1996-97',
       u'SEASON_OPPONENT:atl:1997-98', u'SEASON_OPPONENT:atl:1999-00',
       u'SEASON_OPPONENT:atl:2000-01', u'SEASON_OPPONENT:atl:2001-02',
       u'SEASON_OPPONENT:atl:2002-03', u'SEASON_OPPONENT:atl:2003-04',
       u'SEASON_OPPONENT:atl:2004-05', u'SEASON_OPPONENT:atl:2005-06',
       u'SEASON_OPPONENT:atl:2006-07', u'SEASON_OPPONENT:atl:2007-08',
       u'SEASON_OPPONENT:atl:2008-09', u'SEASON_OPPONENT:atl:2009-10',
       u'SEASON_OPPONENT:atl:2010-11', u'SEASON_OPPONENT:atl:2011-12',
       u'SEASON_OPPONENT:atl:2012-13', u'SEASON_OPPONENT:atl:2013-14',
       u'SEASON_OPPONENT:atl:2014-15'],
      dtype='object')


---

### Make predictor and target variables. Normalize the predictors.

Why is normalization necessary for regularized regressions?

There is a class in sklearn.preprocessing called `StandardScaler`. Look it up and figure out how to use it to normalize your predictor matrix. 

In [5]:
y = kobe.SHOTS_MADE.values
X = kobe.iloc[:,1:]
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

ss.fit(X)
Xn = ss.transform(X)


In [6]:
y = kobe.SHOTS_MADE.values
X = kobe.iloc[:,1:]

# Initialize the StandardScaler object


ss = StandardScaler() 
 
# use the "fit_transform" function to normalize the X design matrix
Xn = ss.fit_transform(X)

Xnorm = pd.DataFrame(Xn, columns = X.columns)
Xnorm

# Normalization is necessary for regularized regression because the beta
# values for each predictor variable must be on the same scale. If betas
# are different sizes just because of the scale of predictor variables
# the regularization term can't determine which betas are more/less 
# important based on their size.

X.describe()
Xnorm.describe()

,AWAY_GAME,SEASON_OPPONENT:atl:1996-97,SEASON_OPPONENT:atl:1997-98,SEASON_OPPONENT:atl:1999-00,SEASON_OPPONENT:atl:2000-01,SEASON_OPPONENT:atl:2001-02,SEASON_OPPONENT:atl:2002-03,SEASON_OPPONENT:atl:2003-04,SEASON_OPPONENT:atl:2004-05,SEASON_OPPONENT:atl:2005-06,...,ACTION_TYPE:tip_layup_shot,ACTION_TYPE:tip_shot,ACTION_TYPE:turnaround_bank_shot,ACTION_TYPE:turnaround_fadeaway_bank_jump_shot,ACTION_TYPE:turnaround_fadeaway_shot,ACTION_TYPE:turnaround_finger_roll_shot,ACTION_TYPE:turnaround_hook_shot,ACTION_TYPE:turnaround_jump_shot,SEASON_GAME_NUMBER,CAREER_GAME_NUMBER
count,1.558000e+03,1.558000e+03,1.558000e+03,1.558000e+03,1.558000e+03,1.558000e+03,1.558000e+03,1.558000e+03,1.558000e+03,1.558000e+03,...,1.558000e+03,1.558000e+03,1.558000e+03,1.558000e+03,1.558000e+03,1.558000e+03,1.558000e+03,1.558000e+03,1.558000e+03,1.558000e+03
mean,2.365815e-17,1.368182e-17,1.368182e-17,1.368182e-17,9.121216e-18,9.121216e-18,9.121216e-18,1.368182e-17,1.368182e-17,1.482198e-17,...,-4.560608e-18,-4.560608e-17,-1.824243e-17,9.121216e-18,1.824243e-16,-2.280304e-18,-9.121216e-18,7.296973e-17,9.121216e-17,-1.459395e-16
std,1.000321e+00,1.000321e+00,1.000321e+00,1.000321e+00,1.000321e+00,1.000321e+00,1.000321e+00,1.000321e+00,1.000321e+00,1.000321e+00,...,1.000321e+00,1.000321e+00,1.000321e+00,1.000321e+00,1.000321e+00,1.000321e+00,1.000321e+00,1.000321e+00,1.000321e+00,1.000321e+00
min,-1.001285e+00,-3.585174e-02,-3.585174e-02,-2.534286e-02,-2.534286e-02,-3.585174e-02,-3.585174e-02,-2.534286e-02,-2.534286e-02,-3.585174e-02,...,-3.585174e-02,-2.818064e-01,-1.839222e-01,-2.534286e-02,-3.425907e-01,-3.574600e-02,-8.842809e-02,-6.432184e-01,-1.610867e+00,-1.733044e+00
25%,-1.001285e+00,-3.585174e-02,-3.585174e-02,-2.534286e-02,-2.534286e-02,-3.585174e-02,-3.585174e-02,-2.534286e-02,-2.534286e-02,-3.585174e-02,...,-3.585174e-02,-2.818064e-01,-1.839222e-01,-2.534286e-02,-3.425907e-01,-3.574600e-02,-8.842809e-02,-6.432184e-01,-8.428132e-01,-8.653954e-01
50%,9.987171e-01,-3.585174e-02,-3.585174e-02,-2.534286e-02,-2.534286e-02,-3.585174e-02,-3.585174e-02,-2.534286e-02,-2.534286e-02,-3.585174e-02,...,-3.585174e-02,-2.818064e-01,-1.839222e-01,-2.534286e-02,-3.425907e-01,-3.574600e-02,-8.842809e-02,-6.432184e-01,-3.635684e-02,2.996766e-05
75%,9.987171e-01,-3.585174e-02,-3.585174e-02,-2.534286e-02,-2.534286e-02,-3.585174e-02,-3.585174e-02,-2.534286e-02,-2.534286e-02,-3.585174e-02,...,-3.585174e-02,-2.818064e-01,-1.839222e-01,-2.534286e-02,-3.425907e-01,-3.574600e-02,-8.842809e-02,4.224971e-01,7.700996e-01,8.654553e-01
max,9.987171e-01,2.789265e+01,2.789265e+01,3.945884e+01,3.945884e+01,2.789265e+01,2.789265e+01,3.945884e+01,3.945884e+01,2.789265e+01,...,2.789265e+01,1.106839e+01,1.179686e+01,3.945884e+01,1.257104e+01,2.995240e+01,2.258712e+01,6.503345e+00,2.383012e+00,1.730881e+00


---

### Build a linear regression predicting `SHOTS_MADE` from the rest of the columns.

Cross-validate the $R^2$ of a linear regression model with 10 cross-validation folds.

How does it perform?

In [15]:
linreg = LinearRegression()
#vanilla linear regression
linreg_scores = cross_val_score(linreg, Xn, y, cv=5)
# linreg.fit(Xn, y)
# linreg.predict(Xn)
# print "score with normalization"  + str(linreg.score(Xn, y))

# linreg = LinearRegression()
# linreg.fit(X, y)
# linreg.predict(X)
# print "score without normalization" + str(linreg.score(X, y))
print linreg_scores
print np.mean(linreg_scores)

[ -6.87476575e+28  -4.45913757e+28  -2.44014351e+28  -1.68743375e+28
  -3.53241289e+28]
-3.79877869439e+28


In [ ]:
# The mean R^2 is extremely negative. All the R^2 scores are negative in crossvalidation.
# The linear regression is performing far worse than baseline on the test sets.
# It is probably dramatically overfitting and the redundant variables are affecting
# the coefficients in weird ways.

---

### Find an optimal value for Ridge regression alpha using RidgeCV

[Go to the documentation and read how RidgeCV works.](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html)

Hint: once the RidgeCV is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.

Recall that Ridge performs best searching alphas through logarithmic space (`np.logspace`).


In [12]:
ridge_alphas = np.logspace(0, 4, 100)

optimal_ridge = RidgeCV(alphas=ridge_alphas, cv=5)
optimal_ridge.fit(Xn, y)

print optimal_ridge.alpha_



559.081018251


---

### Cross-validate the Ridge $R^2$ with the optimal alpha.

Is it better than the Linear regression? If so, why would this be?

In [16]:
ridge = Ridge(alpha=optimal_ridge.alpha_)

ridge_scores = cross_val_score(ridge, Xn, y, cv=5)

print ridge_scores
print np.mean(ridge_scores)

[ 0.68634538  0.56864065  0.535825    0.46270052  0.4989228 ]
0.550486870073


In [11]:
# It's vastly better than the Linear Regression. 
# There is likely so much multicollinearity in the data that vanilla regression
# can't do anything properly. Ridge is able to manage the multicollinearity
# and get a good result.

---

### Find an optimal value for Lasso regression alpha using LassoCV

[Go to the documentation and read how LassoCV works.](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html) It is very similar to RidgeCV.

Hint: again, once the LassoCV is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.

Recall that Lasso, unlike Ridge, performs best searching alphas through linear space (`np.linspace`). However, you can actually let the LassoCV decide itself what alphas to use by instead setting the keyword argument `n_alphas=` to however many alphas you want it to search over.

In [18]:
optimal_lasso = LassoCV(n_alphas=100, cv=5, verbose=1)
optimal_lasso.fit(Xn, y)

print optimal_lasso.alpha_

....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    5.9s finished


0.0565473155307


---

### Cross-validate the Lasso $R^2$ with the optimal alpha.

Is it better than the Linear regression? Is it better than Ridge? For each, why would this be?

Depending on which $R^2$ is better between the Ridge and Lasso, what can you infer about the primary issue in the data?

In [21]:
lasso = Lasso(alpha=optimal_lasso.alpha_)

lasso_scores = cross_val_score(lasso, Xn, y, cv=5)

print lasso_scores
print np.mean(lasso_scores)



[ 0.69484606  0.58808901  0.55314667  0.49096439  0.53620671]
0.572650566506
None


In [14]:
# The lasso performs slightly better than the Ridge, but similarly.
# Lasso deals primarily with the feature selection of valuable variables,
# eliminating ones that are not useful. This also takes care of multicollinearity,
# but in a different way: it will choose the "best" of the correlated variables
# and zero-out the other redundant ones.
# There may also be useless variables in the data which it is simply getting rid
# of entirely.

---

### Look at the coefficients for variables in the Lasso.

1. Show the coefficient for variables, ordered from largest to smallest coefficient by absolute value.
2. What percent of the variables in the original dataset are "zeroed-out" by the lasso?
3. What are the most important predictors for how many shots kobe made in a game?

Note: if you only fit the Lasso within cross_val_score, you will have to refit it outside of that
function to pull out the coefficients.

In [32]:
lasso.fit(Xn, y)

Lasso(alpha=0.056547315530653328, copy_X=True, fit_intercept=True,
   max_iter=1000, normalize=False, positive=False, precompute=False,
   random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [33]:


lasso_coefs = pd.DataFrame({'variable':X.columns,
                            'coef':lasso.coef_,
                            'abs_coef':np.abs(lasso.coef_)})

lasso_coefs.sort_values('abs_coef', inplace=True, ascending=False)

lasso_coefs.head()

,abs_coef,coef,variable
579,1.267017,1.267017,COMBINED_SHOT_TYPE:jump_shot
574,0.828362,0.828362,SHOT_TYPE:2pt_field_goal
566,0.469498,0.469498,SHOT_ZONE_BASIC:restricted_area
611,0.283816,-0.283816,ACTION_TYPE:jump_shot
577,0.278567,0.278567,COMBINED_SHOT_TYPE:dunk


In [34]:
print 'Percent variables zeroed out:', np.sum((lasso.coef_ == 0))/float(X.shape[0])

Percent variables zeroed out: 0.304236200257


---

### Find an optimal value for Elastic Net regression alpha using ElasticNetCV

[Go to the documentation and read how LassoCV works.](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html).

Note here that you will be optimizing both the alpha parameter and the l1_ratio:

    alpha: strength of regularization
    l1_ratio: amount of ridge vs. lasso (0 = all ridge, 1 = all lasso)
    
Do not include 0 in the search for l1_ratio: it will not allow it and break!

You can use n_alphas for the alpha parameters instead of setting your own values: highly recommended!

Also - be careful setting too many l1_ratios over cross-validation folds in your search. It can take a very long time if you choose too many combinations and for the most part there are diminishing returns in this data.

In [37]:
l1_ratios = np.linspace(0.01, 1.0, 5)

optimal_enet = ElasticNetCV(l1_ratio=l1_ratios, n_alphas=10, cv=5,
                            verbose=1)
optimal_enet.fit(Xn, y)

print optimal_enet.alpha_
print optimal_enet.l1_ratio_


........................................................................................................................................................................./Users/basilbeirouti/miniconda3/envs/dsienv/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
.................................................................................[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    9.7s finished


0.0565473155307
1.0


---

### Cross-validate the ElasticNet $R^2$ with the optimal alpha and l1_ratio.

How does it compare to the other regularized regressions?

In [38]:
enet = ElasticNet(alpha=optimal_enet.alpha_, l1_ratio=optimal_enet.l1_ratio_)

enet_scores = cross_val_score(enet, Xn, y, cv=10)

print enet_scores
print np.mean(enet_scores)

[ 0.62359856  0.52594988  0.5373581   0.61516083  0.54289104  0.55459934
  0.53103914  0.44676927  0.46403376  0.50827273]
0.534967263327


---

### Plot the residuals for the ridge, lasso, and elastic net on histograms

This is another way to look at the performance of your model.

The tighter the distribution of residuals around zero, the better your model has performed!

In [ ]:
# Need to fit the ElasticNet and Ridge outside of cross_val_score like i did with the ridge
ridge.fit(Xn, y)
enet.fit(Xn, y)
lasso.fit(Xn, y)

In [ ]:
# model residuals:

ridge_resid = y - ridge.predict(Xn)
lasso_resid = y - lasso.predict(Xn)
enet_resid = y - enet.predict(Xn)


In [ ]:
fig, axarr = plt.subplots(1, 3, figsize=(18, 6))

sns.distplot(ridge_resid, bins=50, hist=True, kde=False, 
             color='steelblue', ax=axarr[0], label='Ridge residuals')

sns.distplot(lasso_resid, bins=50, hist=True, kde=False, 
             color='darkred', ax=axarr[1], label='Lasso residuals')

sns.distplot(enet_resid, bins=50, hist=True, kde=False, 
             color='darkgoldenrod', ax=axarr[2], label='ElasticNet residuals')

plt.show()